#Question 1

In [13]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np

class Classifier(object):
    def __init__(self):
        self.w = None
        self.b = None

    def fit(self, x, y, epochs=300, eta=0.01):
        self.w = np.zeros(shape=(x.shape[1], 1))
        self.b = 0
        for _ in range(epochs):
            for idx, ip in enumerate(x):
                ip = ip.reshape(-1, 1)
                target = y[idx]
                pred = (self.w.T @ ip + self.b)      #@ for dot product
                pred = pred.reshape(-1)
                pred = pred[0]
                pred = 1 if self.sigmoid(pred) > 0.5 else -1
                gradient = pred - target
                self.w -= eta * gradient * ip
                self.b -= eta * gradient

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def predict(self, x):
        pred_list = []
        for ip in x:
            ip = ip.reshape(-1)
            pred = self.w.T @ ip + self.b
            pred = pred.reshape(-1)
            pred = pred[0]
            pred = 1 if pred > 0 else -1
            pred_list.append(pred)
        return pred_list




In [62]:
iris_df = load_iris()
X = iris_df["data"]
y = iris_df["target"]
X = (X - X.mean(axis=0)) / X.std(axis=0)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
# clf_1 -> recognise only setosa and non setosa
clf_1 = Classifier()
clf_1.fit(X_train, np.where(y_train == 0, 1, -1))
y_pred = clf_1.predict(X_test)
# clf_2 -> recognise only versicolor and non versicolor
clf_2 = Classifier()
clf_2.fit(X_train, np.where(y_train == 1, 1, -1), epochs=400)
y_pred = clf_2.predict(X_test)
# clf_2 -> virginica only  and non verginica
clf_3 = Classifier()
clf_3.fit(X_train, np.where(y_train == 2, 1, -1))
y_pred = clf_3.predict(X_test)


In [63]:
# Setosa = 1 -1 -1
# virginica = -1 1 -1
# versicolor = -1 -1 1
final_pred_list = []
pred_1_arr = clf_1.predict(X_test)
pred_2_arr = clf_2.predict(X_test)
pred_3_arr = clf_3.predict(X_test)
arr_len = len(pred_1_arr)
for idx in range(arr_len):
    pred_1 = pred_1_arr[idx]
    pred_2 = pred_2_arr[idx]
    pred_3 = pred_3_arr[idx]
    result = f"{pred_1}{pred_2}{pred_3}"
    if result == "1-1-1":
        final_pred_list.append(0)
    elif result == "-11-1":
        final_pred_list.append(1)
    elif result == "-1-11":
        final_pred_list.append(2)
    else:
        final_pred_list.append(np.random.choice([1, 2]))
print(final_pred_list)

[0, 1, 0, 2, 1, 1, 0, 2, 0, 0, 1, 1, 0, 1, 1, 1, 2, 2, 2, 1, 1, 0, 2, 2, 0, 0, 0, 1, 2, 1]


Question 2

PART 1

In [6]:
import pandas as pd
from numpy import log,dot,exp,shape
from sklearn.preprocessing import PolynomialFeatures

In [8]:
csvfile = pd.read_csv("/content/exam6.txt", delimiter = ',')
csvfile.to_csv('/content/exam6.txt')

In [10]:
X=csvfile.iloc[:,:-1]
y=csvfile.iloc[:,-1]

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42)

In [23]:
poly = PolynomialFeatures(degree=6)
X_trf = poly.fit_transform(X_train)
X_tsf=poly.fit_transform(X_test)

In [58]:
class logicRegression:
  def sigmoid(self,z):
      sig = 1/(1+exp(-z))
      return sig
  def initialize(self,X):
      weights = np.zeros((shape(X)[1]+1,1))
      X = np.c_[np.ones((shape(X)[0],1)),X]
      return weights,X
  def fit(self,X,y,alpha=10,iter=1000):
      weights,X = self.initialize(X)
      def cost(theta):
        z = dot(X,theta)
        cost0 = y.T.dot(log(self.sigmoid(z)))
        cost1 = (1-y).T.dot(log(1-self.sigmoid(z)))
        cost = -((cost1 + cost0))/len(y)
      return cost    
      cost_list = np.zeros(iter,)
      for i in range(iter):
         weights = weights - alpha*dot(X.T,self.sigmoid(dot(X,weights))-np.reshape(y,(len(y),1)))
         cost_list[i] = cost(weights)
      self.weights = weights
      return cost_list
  def predict(self,X):
      z = dot(self.initialize(X)[1],self.initialize(X)[0])
      lis = []
      for i in self.sigmoid(z):
          if i>0.5:
             lis.append(1)
          else:
             lis.append(0)
      return lis


In [59]:
obj=logicRegression()
model=obj.fit(X_trf,y_train)

In [60]:
y_predict=obj.predict(X_tsf)

In [ ]:
y_predict

PART2

In [ ]:
#using ridge regularization

In [55]:
def sigmoid(X, theta):
    temp = np.dot(X,theta)
    temp = np.exp(-temp)
    temp = temp + 1
    temp = 1/temp
    return temp

def costfunc(h, y, theta, lam):
    theta = theta[1:,:]
    m = h.shape[0]
    y1 = y.transpose()
    h1 = np.log(h)
    p1 = np.dot(y1,h1)
    h2 = np.log(1-h)
    p2 = np.dot((1- y).transpose(), h2)
    p3 = sum(lam*(theta**2))/(2*m)
    ans = -(p1 +p2)/m + (p3)
    return ans

def gradDescLogisticRegularised(X, y, alpha, niter):
    theta = np.zeros(shape= (X.shape[1],1))
    h = sigmoid(X, theta)
    m = h.shape[0]
    Jhist = np.zeros(shape = (niter,1))
    
    for i in range(0, niter):
        Jhist[i] = costfunc(h,y,theta,0.2)
        temp = theta
        temp = temp*(0.999) -((alpha*np.dot(X.transpose(), h-y))/m)
        temp[0] = theta[0] - ((alpha*np.dot(X.transpose(), h-y))/m)[0]
        theta = temp
        h = sigmoid(X, theta)
        
    return (theta, Jhist)
